In [1]:
!pip install sentence-transformers faiss-cpu transformers gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 43.2 MB/s eta 0:00:00


In [2]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np


In [20]:
docs = docs = [
    "Grilled Chicken Wrap – $4.50 – 35g protein – 400 kcal",
    "Veggie Pasta Bowl – $3.80 – 12g protein – 350 kcal",
    "Beef Burger Combo – $5.20 – 28g protein – 600 kcal",
    "Fruit Smoothie – $2.90 – rich in vitamins, 180 kcal"
    "Tofu Salad Bowl – $4.20 – 15g protein – 300 kcal",
    "Beef Burrito – $5.50 – 30g protein – 600 kcal",
]


In [21]:
model = SentenceTransformer("all-MiniLM-L6-v2")
doc_embeddings = model.encode(docs)

In [22]:
dimension = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(doc_embeddings))

In [24]:
def recommend_meal(query, top_k=2):
    query_vec = model.encode([query])
    D, I = index.search(np.array(query_vec), top_k)
    results = [docs[i] for i in I[0]]
    return results

In [27]:
print("Query: protein-heavy food under $5")
print("Recommendations:")
for r in recommend_meal("protein-heavy food under $5"):
    print("-", r)

print("\nQuery: cheap vegetarian option")
for r in recommend_meal("cheap vegetarian option"):
    print("-", r)

Query: protein-heavy food under $5
Recommendations:
- Beef Burger Combo – $5.20 – 28g protein – 600 kcal
- Beef Burrito – $5.50 – 30g protein – 600 kcal

Query: cheap vegetarian option
- Beef Burger Combo – $5.20 – 28g protein – 600 kcal
- Beef Burrito – $5.50 – 30g protein – 600 kcal


In [28]:
import gradio as gr

iface = gr.Interface(
    fn=recommend_meal,
    inputs=gr.Textbox(label="Ask your meal question:"),
    outputs=gr.Textbox(label="Recommended Meals"),
    title="🍴 Cafeteria Meal Advisor",
    description="Ask things like: *'What should I eat today under $5?'* or *'Give me a protein-rich lunch option.'*"
)

iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e5415909440017dffd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
